# Phylogenetic signal and evolutionary models for cell size using different metrics

In [1]:
library(ape)
library(phytools)
library(caper)
library(geiger)
library(OUwie)

Loading required package: maps

Loading required package: MASS

Loading required package: mvtnorm

Loading required package: corpcor

Loading required package: nloptr

Loading required package: RColorBrewer



## Tree and data

Read phylogenetic tree

In [2]:
tree <- read.tree("../phylogeny/place/fine_all.nwk")
tree


Phylogenetic tree with 5380 tips and 1961 internal nodes.

Tip labels:
  taxid71518, taxid83984, taxid2193, taxid83985, taxid71152, taxid2203, ...
Node labels:
  N1, N5, N18, N51, N79, N119, ...

Rooted; includes branch lengths.

Read data table

In [3]:
data <- read.table("../phylogeny/place/fine_all.tsv", header=TRUE, sep="\t", quote="")
head(data, 3)

,taxid,length,width,volume,surface,shape,species,genus,family,order,⋯,rank,node,genome,gc,proteins,coding,rrnas,MILC,ENCprime,hash
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,taxid11,2.371708,1.0606602,1.7831870,7.902917,rod-shaped,Cellulomonas gilvus,Cellulomonas,Cellulomonadaceae,Micrococcales,⋯,species,G000218545,3526441,73.81,3206,91.77278,2,-0.2620050,0.10083562,1.88
2,taxid14,10.000000,0.4898979,1.8541744,15.390598,rod-shaped,Dictyoglomus thermophilum,Dictyoglomus,Dictyoglomaceae,Dictyoglomales,⋯,species,G000020965,1959987,33.74,1890,93.77725,2,-0.0644294,0.03020484,0.58
3,taxid23,1.500000,0.7000000,0.4874705,3.298672,rod-shaped,Shewanella colwelliana,Shewanella,Shewanellaceae,Alteromonadales,⋯,species,G000518705,4575622,45.39,4094,87.38314,0,-0.6533632,0.24898652,1.25


Calculate some metrics

In [4]:
data[[paste("vsratio")]] = (data[['volume']] / data[['surface']])

In [5]:
data[[paste("svratio")]] = (data[['surface']] / data[['volume']])

In [6]:
data[[paste("equivalent_spherical")]] = ((6 * data[['volume']]) / pi) ^ (1/3)

In [7]:
data[[paste("unscaled_volume")]] = (data[['volume']]) ^ (1/3)

In [8]:
data[[paste("unscaled_surface")]] = (data[['surface']]) ^ (1/3)

In [9]:
metrics = c('length', 'width', 'unscaled_volume', 'unscaled_surface', 'svratio', 'vsratio',
            'equivalent_spherical', 'volume', 'surface')

Log transform metrics

In [10]:
for (m in metrics) {
    data[[paste("log", m, sep="_")]] = log10(data[[m]])
}

In [11]:
metrics = c('log_length', 'log_width', 'log_unscaled_volume', 'log_unscaled_surface',
            'log_svratio', 'log_vsratio',
            'log_equivalent_spherical', 'log_volume', 'log_surface')

## Phylogenetic signal: Pagel's $\lambda$ and Bloomberg's K

In [12]:
# Binarize tree - required for geiger package
tree2 <- multi2di(tree)

In [13]:
# Dataframe to save outputs
df_geiger <- data.frame(matrix(ncol = 6, nrow = 0))
colnames(df_geiger) <- c('lambda', 'pval0', 'pval1', 'sigsq',
                 'sigsq_unb', 'lnL')
df_geiger

lambda,pval0,pval1,sigsq,sigsq_unb,lnL
<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>


In [14]:
lambda_sigsq <- function(tree, x){
    # Fit model
    lambda2 <- fitContinuous(tree, x, model = "lambda")
    # Reescale tree such that lambda = 0, i.e. star phylogeny
    t0 <- rescale(tree, 'lambda', 0)
    # Fit tree and trait to a BM model of evolution
    lambda_L0 <- fitContinuous(t0, x, model = 'BM')
    # Likelihood ratio test
    LLR0 <- -2 * (lambda_L0$opt$lnL - lambda2$opt$lnL)
    # Get a p-value from a chi-sq distribution
    pval0 <- pchisq(LLR0, df = 1, lower.tail = FALSE)
    
    # Test if lambda != 1
    lambda_L1 <- fitContinuous(tree, x, model = 'BM')
    # Likelihood ratio test
    LLR1 <- -2 * (lambda_L1$opt$lnL - lambda2$opt$lnL)
    # Get a p-value
    pval1 <- pchisq(LLR1, df = 1, lower.tail = FALSE)
    # sigsq
    sigsq <- lambda2$opt$sigsq
    # Get the unbiased estimator of sigsq
    sigsq_unb <- lambda2$opt$sigsq * length(tree$tip.label)/(length(tree$tip.label)-1)
    # Get the likelihood
    lnL <- lambda2$opt$lnL
    
    return(list(lambda = lambda2$opt$lambda, pval0 = pval0, 
                pval1 = pval1, sigsq = sigsq, sigsq_unb = sigsq_unb,
               lnL = lnL))
}

In [15]:
for (m in metrics) {
    print(m)
    datum <- setNames(data[[m]], data$taxid)
    l_sq <- lambda_sigsq(tree2, datum)
    df_geiger[nrow(df_geiger) + 1,] <- c(l_sq$lambda, l_sq$pval0, l_sq$pval1, l_sq$sigsq,
                                    l_sq$sigsq_unb, l_sq$lnL)
    rownames(df_geiger)[nrow(df_geiger)] <- m
}

[1] "log_length"
[1] "log_width"
[1] "log_unscaled_volume"
[1] "log_unscaled_surface"
[1] "log_svratio"
[1] "log_vsratio"
[1] "log_equivalent_spherical"
[1] "log_volume"
[1] "log_surface"


In [16]:
df_geiger

,lambda,pval0,pval1,sigsq,sigsq_unb,lnL
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
log_length,0.8959288,0,2.551824e-81,0.18584154,0.18587609,180.9122
log_width,0.8416497,0,4.716602e-117,0.09463752,0.09465511,1591.7989
log_unscaled_volume,0.8329569,0,5.925951e-128,0.07613246,0.07614661,2119.0055
log_unscaled_surface,0.8484166,0,3.590505e-118,0.03796813,0.03797518,4094.9590
log_svratio,0.8296685,0,3.891426e-129,0.08086371,0.08087874,1935.2597
log_vsratio,0.8296685,0,3.891426e-129,0.08086371,0.08087874,1935.2597
log_equivalent_spherical,0.8329620,0,5.925950e-128,0.07613377,0.07614792,2119.0055
log_volume,0.8329569,0,5.925951e-128,0.68519211,0.68531949,-3791.5286
log_surface,0.8484166,0,3.590505e-118,0.34171313,0.34177666,-1815.5751


In [17]:
write.table(df_geiger, "physig/lambda-sigsq.tsv", sep = "\t", quote = FALSE)

Blomberg's K

In [18]:
df_k <- data.frame(matrix(ncol = 2, nrow = 0))
colnames(df_k) <- c("K", "P")

In [19]:
for (m in metrics) {
    set.seed(42)
    datum <- setNames(data[[m]], data$taxid)
    K <- phylosig(tree2, datum, method = "K", test = TRUE)
    df_k[nrow(df_k) + 1,] <- c(K$K, K$P)
    rownames(df_k)[nrow(df_k)] <- m
}

## Evolutionary models

In [20]:
# Dataframe to save outputs
df_models <- data.frame(matrix(ncol=6, nrow=0))
colnames(df_models) <- c('sigma', 'zo', 'parameter', 'lnl', 'aic', 'aic_weight')
df_models

sigma,zo,parameter,lnl,aic,aic_weight
<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>


In [ ]:
startTime <- Sys.time()
for (m in metrics) {
    print(m)
    datum <- setNames(data[[m]], data$taxid)
    # Models of evolution
    bm <- fitContinuous(tree2, datum, model = 'BM', control = list(method = c("subplex","L-BFGS-B"),
    niter = 100, FAIL = 1e+200, hessian = FALSE, CI = 0.95))
    eb <- fitContinuous(tree2, datum, model = 'EB', control = list(method = c("subplex","L-BFGS-B"),
    niter = 100, FAIL = 1e+200, hessian = FALSE, CI = 0.95))
    wh <- fitContinuous(tree2, datum, model = 'white', control = list(method = c("subplex","L-BFGS-B"),
    niter = 100, FAIL = 1e+200, hessian = FALSE, CI = 0.95))
    ou <- fitContinuous(tree2, datum, model = 'OU', ncores = 24, bounds = list(alpha = c(0, 500)))
    # Akaike weigths
    aic_cs <- setNames(c(AIC(bm), AIC(eb), AIC(wh), AIC(ou)), c('BM', 'EB', 'WH', 'OU'))
    aic_cs.w <- aic.w(aic_cs)
    # Add to table
    # BM
    df_models[nrow(df_models) + 1,] <- c(bm$opt$sigsq, bm$opt$z0, '', bm$opt$lnL, bm$opt$aic, aic_cs.w[1])
    rownames(df_models)[nrow(df_models)] <- paste(c(m, 'bm'), collapse = '_')
    # EB
    df_models[nrow(df_models) + 1,] <- c(eb$opt$sigsq, eb$opt$z0, eb$opt$a, eb$opt$lnL, eb$opt$aic, aic_cs.w[2])
    rownames(df_models)[nrow(df_models)] <- paste(c(m, 'eb'), collapse = '_')
    # WH
    df_models[nrow(df_models) + 1,] <- c(wh$opt$sigsq, wh$opt$z0, '', wh$opt$lnL, wh$opt$aic, aic_cs.w[3])
    rownames(df_models)[nrow(df_models)] <- paste(c(m, 'wh'), collapse = '_')
    # OU
    df_models[nrow(df_models) + 1,] <- c(ou$opt$sigsq, ou$opt$z0, ou$opt$alpha, ou$opt$lnL, ou$opt$aic, aic_cs.w[4])
    rownames(df_models)[nrow(df_models)] <- paste(c(m, 'ou'), collapse = '_')
}
endTime <- Sys.time()
print(endTime - startTime)

[1] "log_length"


Warning message in fitContinuous(tree2, datum, model = "OU", ncores = 24, bounds = list(alpha = c(0, :
“Non-ultrametric tree with OU model, using VCV method.”
Warning message in cache$dat - mu:
“Recycling array of length 1 in vector-array arithmetic is deprecated.
  Use c() or as.vector() instead.
”
Warning message in cache$dat - mu:
“Recycling array of length 1 in vector-array arithmetic is deprecated.
  Use c() or as.vector() instead.
”


[1] "log_width"


Warning message in fitContinuous(tree2, datum, model = "OU", ncores = 24, bounds = list(alpha = c(0, :
“Non-ultrametric tree with OU model, using VCV method.”
Warning message in cache$dat - mu:
“Recycling array of length 1 in vector-array arithmetic is deprecated.
  Use c() or as.vector() instead.
”
Warning message in cache$dat - mu:
“Recycling array of length 1 in vector-array arithmetic is deprecated.
  Use c() or as.vector() instead.
”


[1] "log_unscaled_volume"


Warning message in fitContinuous(tree2, datum, model = "OU", ncores = 24, bounds = list(alpha = c(0, :
“Non-ultrametric tree with OU model, using VCV method.”


In [ ]:
df_models

In [ ]:
write.table(df_models, "physig/evol_models.tsv", sep = "\t", quote = FALSE)